In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset

In [ ]:
dataset.columns

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset.describe()

In [ ]:
dataset['region'].value_counts()

In [ ]:
plt.plot(dataset['expenses'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
dataset['sex'] = label_encoder.fit_transform(dataset['sex'])

In [ ]:
dataset['smoker'] = label_encoder.fit_transform(dataset['smoker'])
dataset['region'] = label_encoder.fit_transform(dataset['region'])

In [ ]:
dataset

In [ ]:
test_dataset = dataset.sample(frac=0.2)
len(test_dataset)

In [ ]:
test_labels = test_dataset['expenses']
test_dataset = test_dataset.drop(columns = ['expenses'])

In [ ]:
test_labels

In [ ]:
train_dataset = dataset[~dataset.isin(test_dataset)].dropna()
len(train_dataset)

In [ ]:
X = train_dataset.drop(columns=['expenses'])

In [ ]:
Y = train_dataset['expenses']

In [ ]:
X

In [ ]:
Y

In [ ]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
X_train = ss.fit_transform(X)
test_dataset = ss.fit_transform(test_dataset)

In [ ]:
X_train.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, Y)

In [ ]:
y_pred3 = rf.predict(test_dataset)

In [ ]:
from sklearn import metrics
s3 = metrics.mean_absolute_error(test_labels, y_pred3)
s3

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

# Adjusted learning rate
optimizer = Adam(learning_rate==0.001)

model.compile(optimizer=optimizer, loss='mean_absolute_error', metrics=['mae','mse'])

# Train the model with more epochs
model.fit(X_train, Y, epochs=50, batch_size=32, validation_split=0.2)


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
